In [1]:
from aocd import get_data

# Day 12: Garden Groups

## Part One

The goal of this problem is to determine the total price of fencing regions in a garden plot map. Each region is a group of contiguous (horizontally or vertically connected) garden plots of the same type.

For each region:

1. Area is the number of plots in the region.
2. Perimeter is the number of sides of the plots on the region's boundary that are not adjacent to other plots of the same type.
3. The price of fencing a region is given by:<br/>
`Price = Area × Perimeter`<br/>
4. The total price is the sum of the prices for all regions in the garden plot map.


Steps:

1. Identify Regions:<br/>
  a. Traverse the garden plot map to identify contiguous regions of the same plant type.<br/>
  b. Use DFS (Depth-First Search) or BFS (Breadth-First Search) to explore and mark each region.<br/>

2. Calculate Area:<br/>
  a. For each region, count the number of garden plots (this is the area).<br/>

3. Calculate Perimeter:<br/>
  a. For each plot in a region, check its neighbors.<br/>
  b. If a side of the plot touches the boundary of the map or a different plant type, it contributes to the perimeter.<br/>

4. Calculate Total Price:<br/>
  a. For each region, calculate the price (area × perimeter).<br/>
  b. Sum up the prices of all regions.<br/>

In [2]:
import numpy as np

In [3]:
def is_in_bounds(cell, grid):
    row, col = cell
    rows, cols = grid.shape
    return 0 <= row < rows and 0 <= col < cols

In [4]:
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def get_neighbours(cell, grid, directions=directions, include_edges=False):
    """
    Return the adjacent cells (up, down, left, right) for a given cell
    """
    row, col = cell

    neighbours = []
    for dr, dc in directions:
        new_cell = (row + dr, col + dc)
        if is_in_bounds(new_cell, grid):
            neighbours.append(new_cell)
        elif include_edges:
            # if it's the edge return empty cell
            neighbours.append((-1, -1))

    return neighbours

In [5]:
def is_empty(cell):
    return -1 in cell

In [6]:
def explore_region(grid, start_cell, visited, verbose=False):
    """
    Use depth-first search to find all connected cells of the same plant type
    Keep track of visited cells to avoid revisiting
    """
    stack = [start_cell]
    region = []
    plant_type = grid[start_cell]
    if verbose:
        print(f"Exploring region {plant_type}...")
    
    while stack:  # while stack not empty
        current = stack.pop()
        if current not in visited:
            visited.add(current)
            region.append(current)
            for neighbour in get_neighbours(current, grid):
                if grid[neighbour] == plant_type:
                    stack.append(neighbour)

    return region, visited

In [7]:
def calculate_perimeter(grid, region_cells):
    """
    For each cell in a region, check its neighbouring sides
    Increase the perimeter count if the side touches the boundary or a different plant type
    """
    perimeter = 0
    
    for cell in region_cells:
        region_type = grid[cell]
        # get neighbours doesn't return anything if we're at the edge of the grid
        for neighbour in get_neighbours(cell, grid, include_edges=True):
            if is_empty(neighbour):
                perimeter += 1
            elif grid[neighbour] != region_type:
                perimeter += 1

        rows, cols = grid.shape
        
    return perimeter

In [8]:
def calculate_total_fencing_price(grid, verbose=False):
    """
    Loop through each cell in the garden map
    Use explore_region to identify regions and their areas
    Calculate the perimeter for each region and compute the price
    Sum up the total price for all regions
    """
    visited = set()
    total_price = 0

    for cell in np.ndindex(grid.shape):
        if cell not in visited:
            region_cells, visited = explore_region(grid, cell, visited, verbose)
            if verbose:
                print(f"{region_cells=}")
            area = len(region_cells)
            perimeter = calculate_perimeter(grid, region_cells)
            if verbose:
                print(f"{perimeter=}")
            price = area * perimeter
            total_price += price

    return total_price

In [9]:
example_input = """
AAAA
BBCD
BBCC
EEEC
"""

In [10]:
lines = example_input.strip().split('\n')
matrix = np.array([list(line) for line in lines]); matrix

array([['A', 'A', 'A', 'A'],
       ['B', 'B', 'C', 'D'],
       ['B', 'B', 'C', 'C'],
       ['E', 'E', 'E', 'C']], dtype='<U1')

In [11]:
calculate_total_fencing_price(matrix, verbose=True)

Exploring region A...
region_cells=[(0, 0), (0, 1), (0, 2), (0, 3)]
perimeter=10
Exploring region B...
region_cells=[(1, 0), (1, 1), (2, 1), (2, 0)]
perimeter=8
Exploring region C...
region_cells=[(1, 2), (2, 2), (2, 3), (3, 3)]
perimeter=10
Exploring region D...
region_cells=[(1, 3)]
perimeter=4
Exploring region E...
region_cells=[(3, 0), (3, 1), (3, 2)]
perimeter=8


140

In [12]:
example_input_2 = """
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
"""

In [13]:
lines = example_input_2.strip().split('\n')
matrix = np.array([list(line) for line in lines]); matrix

array([['O', 'O', 'O', 'O', 'O'],
       ['O', 'X', 'O', 'X', 'O'],
       ['O', 'O', 'O', 'O', 'O'],
       ['O', 'X', 'O', 'X', 'O'],
       ['O', 'O', 'O', 'O', 'O']], dtype='<U1')

In [14]:
calculate_total_fencing_price(matrix, verbose=True)

Exploring region O...
region_cells=[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (2, 3), (2, 2), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (3, 4), (3, 2), (1, 0), (1, 2)]
perimeter=36
Exploring region X...
region_cells=[(1, 1)]
perimeter=4
Exploring region X...
region_cells=[(1, 3)]
perimeter=4
Exploring region X...
region_cells=[(3, 1)]
perimeter=4
Exploring region X...
region_cells=[(3, 3)]
perimeter=4


772

In [15]:
data = get_data(day=12, year=2024)

In [16]:
lines = data.strip().split('\n')
matrix = np.array([list(line) for line in lines]); matrix

array([['U', 'U', 'U', ..., 'J', 'J', 'J'],
       ['U', 'U', 'K', ..., 'J', 'J', 'J'],
       ['U', 'U', 'K', ..., 'J', 'J', 'J'],
       ...,
       ['S', 'S', 'S', ..., 'H', 'H', 'H'],
       ['S', 'S', 'E', ..., 'H', 'H', 'H'],
       ['S', 'S', 'S', ..., 'H', 'H', 'H']], dtype='<U1')

In [17]:
calculate_total_fencing_price(matrix, verbose=False)

1363484

## Part Two

With the new addition of "bulk pricing", the price for fencing a region is calculated as:
`Price = Area × Number of Sides`

So we need a new function to calculate the number of sides in a region.

In [80]:
def region_sides(region_cells):
    region = np.array(region_cells)
    print(region)
    edges = 0
    
    for r in set(region[:, 0]):
        print(f"{r=}")
        rows = region[region[:, 0] == r]
        edges += find_consecutive_groups(rows[:, 1])

    for c in set(region[:, 1]):
        print(f"{c=}")
        cols = region[region[:, 1] == c]
        edges += find_consecutive_groups(cols[:, 0])
            
    return edges
    

def find_consecutive_groups(arr):
    print(f"{arr=}")
    groups = []
    current_group = [arr[0]]

    for i in range(1, len(arr)):
        if arr[i] == arr[i - 1] + 1:
            current_group.append(arr[i])
        else:
            groups.append(current_group)
            current_group = [arr[i]]

    groups.append(current_group)
    
    return len(groups)

In [81]:
def calculate_bulk_pricing_fencing_price(grid, verbose=False):
    """
    Loop through each cell in the garden map
    Use explore_region to identify regions and their areas
    Calculate the *num unique sides* for each region and compute the price
    Sum up the total price for all regions
    """
    visited = set()
    total_price = 0

    for cell in np.ndindex(grid.shape):
        if cell not in visited:
            region_cells, visited = explore_region(grid, cell, visited)
            if verbose:
                print(f"{region_cells=}")
            area = len(region_cells)
            unique_sides = region_sides(region_cells)
            if verbose:
                print(f"{unique_sides=}")
            price = area * unique_sides
            total_price += price

    return total_price

In [82]:
lines = example_input.strip().split('\n')
matrix = np.array([list(line) for line in lines]); matrix

array([['A', 'A', 'A', 'A'],
       ['B', 'B', 'C', 'D'],
       ['B', 'B', 'C', 'C'],
       ['E', 'E', 'E', 'C']], dtype='<U1')

In [83]:
calculate_bulk_pricing_fencing_price(matrix, verbose=True)

region_cells=[(0, 0), (0, 1), (0, 2), (0, 3)]
[[0 0]
 [0 1]
 [0 2]
 [0 3]]
r=np.int64(0)
arr=array([0, 1, 2, 3])
c=np.int64(0)
arr=array([0])
c=np.int64(1)
arr=array([0])
c=np.int64(2)
arr=array([0])
c=np.int64(3)
arr=array([0])
unique_sides=5
region_cells=[(1, 0), (1, 1), (2, 1), (2, 0)]
[[1 0]
 [1 1]
 [2 1]
 [2 0]]
r=np.int64(1)
arr=array([0, 1])
r=np.int64(2)
arr=array([1, 0])
c=np.int64(0)
arr=array([1, 2])
c=np.int64(1)
arr=array([1, 2])
unique_sides=5
region_cells=[(1, 2), (2, 2), (2, 3), (3, 3)]
[[1 2]
 [2 2]
 [2 3]
 [3 3]]
r=np.int64(1)
arr=array([2])
r=np.int64(2)
arr=array([2, 3])
r=np.int64(3)
arr=array([3])
c=np.int64(2)
arr=array([1, 2])
c=np.int64(3)
arr=array([2, 3])
unique_sides=5
region_cells=[(1, 3)]
[[1 3]]
r=np.int64(1)
arr=array([3])
c=np.int64(3)
arr=array([1])
unique_sides=2
region_cells=[(3, 0), (3, 1), (3, 2)]
[[3 0]
 [3 1]
 [3 2]]
r=np.int64(3)
arr=array([0, 1, 2])
c=np.int64(0)
arr=array([3])
c=np.int64(1)
arr=array([3])
c=np.int64(2)
arr=array([3])
unique_si

74